In [1]:
## Import Libraries ##
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

from sklearn.feature_extraction import DictVectorizer
%matplotlib inline

## Pull in Data (csv)

In [2]:
fires = pd.read_csv('fires_with_climate.csv')


C:\Users\604906\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (5,11,15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Feature Exploration

In [3]:
fires.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1880465 entries, 0 to 1880464
Data columns (total 18 columns):
FIRE_YEAR           int64
DISCOVERY_DATE      object
DISCOVERY_MONTH     object
DISCOVERY_DOY       int64
DISCOVERY_TIME      float64
CONT_DATE           object
CONT_TIME           float64
STAT_CAUSE_DESCR    object
FIRE_SIZE           float64
FIRE_SIZE_CLASS     object
STATE               object
FIPS_NAME           object
LATITUDE            float64
LONGITUDE           float64
FOD_ID              int64
CLIMATE             object
CLIMATE_ZONE        float64
MOISTURE            object
dtypes: float64(6), int64(3), object(9)
memory usage: 258.2+ MB


In [4]:
fires.head(5)

,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_MONTH,DISCOVERY_DOY,DISCOVERY_TIME,CONT_DATE,CONT_TIME,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,STATE,FIPS_NAME,LATITUDE,LONGITUDE,FOD_ID,CLIMATE,CLIMATE_ZONE,MOISTURE
0,2005,2005-02-02,February,33,1300.0,2005-02-02,1730.0,Miscellaneous,0.10,A,CA,Plumas,40.036944,-121.005833,1,Cold,5.0,B
1,2004,2004-05-12,May,133,845.0,2004-05-12,1530.0,Lightning,0.25,A,CA,Placer,38.933056,-120.404444,2,Hot-Dry,3.0,B
2,2004,2004-05-31,May,152,1921.0,2004-05-31,2024.0,Debris Burning,0.10,A,CA,El Dorado,38.984167,-120.735556,3,Mixed-Dry,4.0,B
3,2004,2004-06-28,June,180,1600.0,2004-07-03,1400.0,Lightning,0.10,A,CA,Alpine,38.559167,-119.913333,4,Cold,6.0,B
4,2004,2004-06-28,June,180,1600.0,2004-07-03,1200.0,Lightning,0.10,A,CA,Alpine,38.559167,-119.933056,5,Cold,6.0,B


In [5]:
SEASON = []

for row in fires['DISCOVERY_MONTH']:
    
    if row in ['March', 'April', 'May']:
        SEASON.append('Spring')
        
    elif row in ['June', 'July', 'August']:
        SEASON.append('Summer')
    
    elif row in ['September','October','November']:
        SEASON.append('Fall')

    else: 
        SEASON.append('Winter')
    

fires['SEASON'] = SEASON
print(fires.head())

   FIRE_YEAR DISCOVERY_DATE DISCOVERY_MONTH  DISCOVERY_DOY  DISCOVERY_TIME  \
0       2005     2005-02-02        February             33          1300.0   
1       2004     2004-05-12             May            133           845.0   
2       2004     2004-05-31             May            152          1921.0   
3       2004     2004-06-28            June            180          1600.0   
4       2004     2004-06-28            June            180          1600.0   

    CONT_DATE  CONT_TIME STAT_CAUSE_DESCR  FIRE_SIZE FIRE_SIZE_CLASS STATE  \
0  2005-02-02     1730.0    Miscellaneous       0.10               A    CA   
1  2004-05-12     1530.0        Lightning       0.25               A    CA   
2  2004-05-31     2024.0   Debris Burning       0.10               A    CA   
3  2004-07-03     1400.0        Lightning       0.10               A    CA   
4  2004-07-03     1200.0        Lightning       0.10               A    CA   

   FIPS_NAME   LATITUDE   LONGITUDE  FOD_ID    CLIMATE  CLIMAT

In [6]:
fires['SEASON'].value_counts()
#Spring 636,120
#Summer 609,252
#Fall 330,694
#Winter 304,399

Spring    636120
Summer    609252
Fall      330694
Winter    304399
Name: SEASON, dtype: int64

In [7]:
fires.isnull().sum()

FIRE_YEAR                0
DISCOVERY_DATE           0
DISCOVERY_MONTH          0
DISCOVERY_DOY            0
DISCOVERY_TIME      882638
CONT_DATE           891531
CONT_TIME           972553
STAT_CAUSE_DESCR         0
FIRE_SIZE                0
FIRE_SIZE_CLASS          0
STATE                    0
FIPS_NAME           678148
LATITUDE                 0
LONGITUDE                0
FOD_ID                   0
CLIMATE             679576
CLIMATE_ZONE        679576
MOISTURE            679886
SEASON                   0
dtype: int64

In [8]:
#new df, rows with any null dropped
fires = fires.dropna()

In [9]:
fires['DISCOVERY_MONTH'].value_counts()

July         76499
April        73320
March        72592
August       68584
May          50855
June         48893
February     45482
September    38063
November     31946
October      31545
January      26714
December     15967
Name: DISCOVERY_MONTH, dtype: int64

In [10]:
#select useful columns
df = fires[['FIRE_YEAR', 'DISCOVERY_MONTH','DISCOVERY_DOY', 'DISCOVERY_TIME', 'CONT_TIME', 'STAT_CAUSE_DESCR', 'FIRE_SIZE', 'FIRE_SIZE_CLASS',
            'LATITUDE', 'LONGITUDE','CLIMATE','CLIMATE_ZONE','MOISTURE','SEASON']]
print(df.head())

   FIRE_YEAR DISCOVERY_MONTH  DISCOVERY_DOY  DISCOVERY_TIME  CONT_TIME  \
0       2005        February             33          1300.0     1730.0   
1       2004             May            133           845.0     1530.0   
2       2004             May            152          1921.0     2024.0   
3       2004            June            180          1600.0     1400.0   
4       2004            June            180          1600.0     1200.0   

  STAT_CAUSE_DESCR  FIRE_SIZE FIRE_SIZE_CLASS   LATITUDE   LONGITUDE  \
0    Miscellaneous       0.10               A  40.036944 -121.005833   
1        Lightning       0.25               A  38.933056 -120.404444   
2   Debris Burning       0.10               A  38.984167 -120.735556   
3        Lightning       0.10               A  38.559167 -119.913333   
4        Lightning       0.10               A  38.559167 -119.933056   

     CLIMATE  CLIMATE_ZONE MOISTURE  SEASON  
0       Cold           5.0        B  Winter  
1    Hot-Dry           3.0    

In [11]:
#Add Burn Time as a new feature
df['BURN_TIME'] = df['CONT_TIME']-df['DISCOVERY_TIME']
print(df.head())

   FIRE_YEAR DISCOVERY_MONTH  DISCOVERY_DOY  DISCOVERY_TIME  CONT_TIME  \
0       2005        February             33          1300.0     1730.0   
1       2004             May            133           845.0     1530.0   
2       2004             May            152          1921.0     2024.0   
3       2004            June            180          1600.0     1400.0   
4       2004            June            180          1600.0     1200.0   

  STAT_CAUSE_DESCR  FIRE_SIZE FIRE_SIZE_CLASS   LATITUDE   LONGITUDE  \
0    Miscellaneous       0.10               A  40.036944 -121.005833   
1        Lightning       0.25               A  38.933056 -120.404444   
2   Debris Burning       0.10               A  38.984167 -120.735556   
3        Lightning       0.10               A  38.559167 -119.913333   
4        Lightning       0.10               A  38.559167 -119.933056   

     CLIMATE  CLIMATE_ZONE MOISTURE  SEASON  BURN_TIME  
0       Cold           5.0        B  Winter      430.0  
1    Hot

C:\Users\604906\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
#check which columns are object types
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 580460 entries, 0 to 1872304
Data columns (total 15 columns):
FIRE_YEAR           580460 non-null int64
DISCOVERY_MONTH     580460 non-null object
DISCOVERY_DOY       580460 non-null int64
DISCOVERY_TIME      580460 non-null float64
CONT_TIME           580460 non-null float64
STAT_CAUSE_DESCR    580460 non-null object
FIRE_SIZE           580460 non-null float64
FIRE_SIZE_CLASS     580460 non-null object
LATITUDE            580460 non-null float64
LONGITUDE           580460 non-null float64
CLIMATE             580460 non-null object
CLIMATE_ZONE        580460 non-null float64
MOISTURE            580460 non-null object
SEASON              580460 non-null object
BURN_TIME           580460 non-null float64
dtypes: float64(7), int64(2), object(6)
memory usage: 70.9+ MB


In [13]:
## lABEL ENCODING ##
# Extract categorical features
categorical_feature_mask = df.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols = df.columns[categorical_feature_mask].tolist()

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
# apply le on categorical feature columns
df[categorical_cols] = df[categorical_cols].apply(lambda col: le.fit_transform(col))

C:\Users\604906\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [14]:
#CHECK
fires.CLIMATE.value_counts(sort=True)


Cold           197407
Mixed-Humid    183689
Hot-Humid      121514
Hot-Dry         38685
Marine          14957
Mixed-Dry       12986
Very-Cold       11222
Name: CLIMATE, dtype: int64

In [15]:
fires.CLIMATE_ZONE.value_counts(sort=True)

3.0    195348
5.0    125284
4.0    113575
6.0     71752
2.0     62804
7.0     11222
1.0       475
Name: CLIMATE_ZONE, dtype: int64

In [16]:
#drop variables that might be correlated with season
df1 = df.drop(['DISCOVERY_MONTH','DISCOVERY_DOY','CLIMATE','CLIMATE_ZONE','MOISTURE'],axis=1)
df1.head(5)

,FIRE_YEAR,DISCOVERY_TIME,CONT_TIME,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,SEASON,BURN_TIME
0,2005,1300.0,1730.0,7,0.10,0,40.036944,-121.005833,3,430.0
1,2004,845.0,1530.0,6,0.25,0,38.933056,-120.404444,1,685.0
2,2004,1921.0,2024.0,3,0.10,0,38.984167,-120.735556,1,103.0
3,2004,1600.0,1400.0,6,0.10,0,38.559167,-119.913333,2,-200.0
4,2004,1600.0,1200.0,6,0.10,0,38.559167,-119.933056,2,-400.0


In [40]:
#spring and summer only
df2 = df1.loc[df1['SEASON'].isin(['1','2'])]
df2.shape #390,743 rows -> confirmed

(390743, 10)

In [41]:
#Train on data from 1992 - 2014
df_train = df2[df2['FIRE_YEAR'] != 2015]
df_train.FIRE_YEAR.describe()

count    361286.000000
mean       2005.529669
std           6.525080
min        1992.000000
25%        2001.000000
50%        2007.000000
75%        2011.000000
max        2014.000000
Name: FIRE_YEAR, dtype: float64

In [42]:
#Test on "future" data: 2015
df_test = df2[df2['FIRE_YEAR'] == 2015]
df_test.FIRE_YEAR.describe()

count    29457.0
mean      2015.0
std          0.0
min       2015.0
25%       2015.0
50%       2015.0
75%       2015.0
max       2015.0
Name: FIRE_YEAR, dtype: float64

## Predicting the SEASON of a Wildfire (Spring or Summer?)

In [44]:
#Try binary classification
X_train, X_test = df_train.drop(['SEASON'], axis=1).values, df_test.drop(['SEASON'], axis=1).values
y_train, y_test = df_train['SEASON'].values, df_test['SEASON'].values

#X = df2.drop(['SEASON'], axis=1).values
#y = df2['SEASON'].values
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 21)

In [45]:
#scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [50]:
#takes a minute to run
classifier = RandomForestClassifier(n_estimators = 50, criterion = 'entropy', random_state = 42)
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [51]:
#check results of actual vs. pred
y_pred = classifier.predict(X_test)
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

#Training on 1992-2014, Test on 2015
#n_estimators = 10; 77.7%
#n_estimators = 20; 78.1%
#n_estimators = 50; 78.6%

#ALL DATA - RANDOM SPLIT
#n_estimators = 10; 81% accuracy
#n_estimators = 20; 81% accuracy
#n_estimators = 50 ; 82% accuracy

Accuracy: 0.7856876124520488
